In [1]:
from Carcassonne_Game.Carcassonne import CarcassonneState
from TicTacToe_Game.TicTacToe import TicTacToeState 
from Function_Optimisation_Game.Function_Optimisation import FunctionOptimisationState
from player.Player import RandomPlayer
from player.MCTS_Player import MCTSPlayer
from player.MCTS_RAVE_Player import MCTS_RAVEPlayer
from player.MCTS_ES_BACK_Player import MCTS_ES_BACK_Player
from player.MCTS_ES_BACK_SEM_Player import MCTS_ES_BACK_SEM_Player

import Experimental_Setup as exps
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from plotly.tools import make_subplots

import os
import pandas as pd
from datetime import date
from collections import OrderedDict
import time
import multiprocessing as mp
import numpy as np
import ast
import random
import math
import statistics as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import bernoulli
from itertools import repeat

In [2]:
#Tictactoe game test

############################################################################
#Experiment parameters
origlogfile=os.path.join("TTT","MCTS")

#Player parameters
iterations = 500
rollouts = 1
lamb = 4
ngen = 20
es_sims = 30
c_param = math.sqrt(2)#[0.5,1,math.sqrt(2),2,3]
seml = 0.1
semu = 0.5
estype = "comma"

#visualization parameters
divisions = 3
buckets = 1000
buckets_2d = 10
divisions_2d=100


#Experiment setup
random_seed = int(c_param*10)
random.seed(random_seed)
np.random.seed(seed=random_seed)
logfile=origlogfile+"ttt_test"


#Players defnition
mcts_player = MCTSPlayer(iterations=iterations
                        ,c_param = c_param
                        ,name='MCTS_c' + str(c_param)
                        ,rollouts = 1
                        ,logs=True
                        ,logfile=logfile)
                        #,random_seed = random_seed)
s_b_mcts_player = MCTS_ES_BACK_SEM_Player(iterations=iterations-(lamb*ngen*es_sims)
                        ,c_param=c_param
                        ,rollouts = 1
                        ,name='SE_MCTS' + str(iterations-(lamb*ngen*es_sims))
                        ,Lambda=lamb
                        ,NGen=ngen
                        ,ES_Sims=es_sims
                        ,ESType=estype
                        ,logs=True
                        ,logfile=logfile
                        ,Sem_L=seml
                        ,Sem_U=semu)
random_player = RandomPlayer()
players = [random_player, mcts_player]#, 

#Execution door to he , agamemnon, andronikus, stradiv, da vinci, zhengde, james arm, queen vict, magna
#state = FunctionOptimisationState(players=[mcts_player], function=func_index, ranges=ranges, splits=branching_factor)# give any single player
state = TicTacToeState()
used_func = state.function
#state = FunctionOptimisationState(players=[random_player], function=func2, ranges=[[0,1],[0,1]], splits=3, minimum_step=0.0000001) #multiple dimensions

all_data = []
all_fo_logs = pd.DataFrame()
for p_id, player in enumerate(players):
      fo_logs, data = exps.multiple_runs(state, used_func, player, runs, random_seed, divisions)
      all_fo_logs = pd.concat([all_fo_logs,fo_logs])
      all_data.append(data)
      all_data_df = pd.concat(all_data)
      all_data_df.to_csv(os.path.join('logs', logfile, "Tree_data.csv"), index=False)
#m2 = s_b_mcts_player.chooseAction(state)
#plot = exps.show_search(all_data, used_func, title, n_buckets = buckets, divisions=divisions)
#plot.write_image(os.path.join('logs', logfile, "Averaged_plot" + '.png'), width=1920, height=1080)
#show_2d_search([data], used_funlbc, title, divisions_2d, n_buckets = buckets_2d)
#show_search([mcts_player,s_b_mcts_player], used_func, title, n_buckets = 50, divisions=4)
#print("Best reward path:",str(stats.mean(ors2)),str(stats.stdev(ors2)))



#Create final logs
#Combine separate logs
exps.CombineFiles(logs=True,logfile=logfile)

#Parameters logs
final_logs={}
final_logs["runs"]=runs
final_logs["random_seed"]=random_seed
final_logs["logfile"]=logfile
#Problem parameters
final_logs["branching_factor"]=branching_factor
final_logs["func_index"]=func_index
final_logs["ranges"]=ranges
#Player parameters
final_logs["iterations"]=iterations
final_logs["lamb"]=lamb
final_logs["ngen"]=ngen
final_logs["es_sims"]=es_sims
final_logs["c_param"]=c_param
final_logs["seml"]=seml
final_logs["semu"]=semu
final_logs["estype"]=estype

final_df = pd.DataFrame(final_logs)
final_df.to_csv(os.path.join('logs', logfile, "Parameter_logs" + '.csv'), index=False)

#Results logs
player_logs = defaultdict(lambda:[])
for player in players:
      temp_logs = all_fo_logs[all_fo_logs["Player"]==player.name]
      player_logs["Player"].append(player.name)
      player_logs["Mean_Tree_Nodes"].append(temp_logs["Tree_Nodes"].mean())
      player_logs["Std_Tree_Nodes"].append(temp_logs["Tree_Nodes"].std())
      player_logs["Mean_Terminals_Reached"].append(temp_logs["Terminals_Reached"].mean())
      player_logs["Std_Terminals_Reached"].append(temp_logs["Terminals_Reached"].std())
      player_logs["Mean_Max_Visits_Path"].append(temp_logs["Max_Visits_Path"].mean())
      player_logs["Std_Max_Visits_Path"].append(temp_logs["Max_Visits_Path"].std())
      player_logs["Mean_Max_Reward_Path"].append(temp_logs["Max_Reward_Path"].mean())
      player_logs["Std_Max_Reward_Path"].append(temp_logs["Max_Reward_Path"].std())
      player_logs["Tree_Reaches_Terminal"].append(temp_logs["Max_Reward_Path"].values.sum()/len(temp_logs))

player_df = pd.DataFrame(player_logs)
player_df.to_csv(os.path.join('logs', logfile, "Final_Player_logs" + '.csv'), index=False)
all_fo_logs.to_csv(os.path.join('logs', logfile, "Player_logs" + '.csv'), index=False)



NameError: name 'os' is not defined

In [ ]:
#plots to compare different algorithms in the same function

def get_subset(data, agent_name, f_index, c_param=None):
   temp_data = data.loc[data["f_index"]==f_index]
   if c_param is not None:
      temp_data = temp_data.loc[temp_data["c_param"]==c_param]
   #print(temp_data["player"].unique())
   temp_data = temp_data.loc[temp_data["player"] == agent_name]
   return temp_data

dummy_state = FunctionOptimisationState(players=[None], function=0, ranges=[[0,1]], splits=2)
functions = dummy_state.function_list
join = "/"
logs_path = "logs/Old/FO"
data = pd.read_csv(logs_path + join + "collective_tree_logs.csv")
n_buckets = 128
f_max_locations = [0.5,0.867,None,0.1,0.1]

for f_index in [0,1,2,3,4]:
   agents_names = data["player"].unique()
   print(agents_names)
   c_params = data["c_param"].unique()
   c_params.sort()
   generic_name = "MCTS_c"
   agents_names = [x for x in agents_names]
   names = []
   data_list = []

   #order names
   for c in c_params:
      string_c = str(c)
      if string_c.split(".")[-1] == "0":
            string_c = string_c[:-2]
      names.append(generic_name + string_c)
   for name in agents_names:
      if "SE_MCTS" in name:
            names.append(name)
   for name in names:
      temp_data = get_subset(data, name, f_index)
      data_list.append(temp_data)
   
   treated_names = []
   for it,st in enumerate(names):
      new_string = st.replace("_c"," C = ")
      new_string = new_string.replace("1.414","\u221A\u03052\u0305fred")
      new_string = new_string.split("fred")[0]
      if "SE_MCTS" in new_string:
            if "2600" in new_string:
               new_string = "SIEA_MCTS 2600 iterations"
            else:
               new_string = "SIEA_MCTS 5000 iterations"
      treated_names.append(new_string)
   #plot = exps.show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, subplot_titles = treated_names+["Function "+str(f_index+1)], max_x_location=f_max_locations[f_index], y_ref_value=None)
   plot = exps.show_search(data_list, functions[f_index], "Exploration and exploitation distribution for Function " + str(f_index+1) + "." , 3, n_buckets = n_buckets, subplot_titles = ["Function "+str(f_index+1)] + treated_names, max_x_location=f_max_locations[f_index], y_ref_value=None)
   plot.write_image(os.path.join(logs_path, "F" + str(f_index+1) + "_results"+ str(n_buckets) + '.png'))#, width=800, height=800)
   plot.show()

In [ ]:
#Plots to compare the number of bins
bins_list = [2**6,2**7,2**8,2**9,2**10]
f_list = [0,1,2,3,4]
agent_name_part = "SE_MCTS"
#agent_name_part = "SE_MCTS2600"
#agent_name_part = "MCTS_c1.4142135623730951"
agent_name = None

for a_n in data["player"].unique():
   if agent_name_part == a_n:
      agent_name = a_n
      print(agent_name)
      break

for f_index in f_list:
   data_list = []
   temp_data = get_subset(data, agent_name, f_index)
   for _ in range(len(bins_list)):
      data_list.append(temp_data)
   plot = exps.show_search(data_list, functions[f_index], agent_name, 3, subplot_titles = [str(nb)+" bins" for nb in bins_list], n_buckets = bins_list, max_x_location=f_max_locations[f_index])
   plot.show()
   #plot.write_image(os.path.join(logs_path, "Bins_comparison_f" + str(f_index+1) + "_agent_"+ agent_name + '.png'))#, width=800, height=800)

In [32]:
def show_search_depth(data_list, plot_title, data_distance = 20, markers_distance = 20, gray_scale=False):
   #agents need to be re-ordered (no hardcode order)
   if gray_scale:
      colors = ["#696969","#4f4f4f","#2f2f2f","#000000","#0f0f0f","#666666","#234654"]

   else:
      colors = ["#820263"
                , "#4F5D2F"
                ,  "#000000"
                ,"#F6511D"
                , "#C874D9"#"#9B6DC6 "
                ,  "#355691" ]
   colors = ["#000000"
                , "#B10909"
                ,  "#5B8C5A"
                ,"#56638A"
                , "#EC7316"#"#9B6DC6 "
                ,  "#FC738C" ]


   markers = ["diamond","cross","x","triangle-up","triangle-down","star","star-square","corcle-cross","square", "pentagon"]
   marker_padding = int(markers_distance/len(data_list))

   fig = go.Figure()
   max_max_y = 0
   for data_idx, data in enumerate(data_list):#list(reversed(data_list)):
      name = data["player"].unique()[0]
      #if len(name) > 12: name = name[:12]
      #print(len(data))
      #temp_data = data.groupby(by="id").mean()
      #print(len(temp_data))
      x = data["id"].unique()
      x = [k for k in x if k%data_distance==0]
      #y = [stats.mean(data.loc[data["id"]==id,["feature_0"]]["feature_0"]) for id in x]
      y=[]
      for id in x:#list(reversed(x)):
         temp = data.loc[data["id"]==id,["feature_0"]]["feature_0"]
         #print("filtered_len",len(temp))
         y.append(stats.mean(temp))

      max_y = max(y)
      if max_y > max_max_y:
         max_max_y = max_y

      x_marker = []
      y_marker = []
      for count in range(len(x)):
         if count%(markers_distance+data_idx*marker_padding) == 0:
            x_marker.append(x[count])
            y_marker.append(y[count])

      fig.add_trace(go.Scatter(x=x, y=y
            ,showlegend=False
            ,name=name
            ,mode="lines"#+markers'#"markers"
            ,marker_symbol = markers[data_idx]# + "-open-dot"
                        #,marker_color = "black"
                        #,marker_size=9
                        ,marker=dict(
                            color=colors[data_idx]#"black",#"red",#'rgba(135, 206, 250, 0.5)',
                            ,size=9
                            ,line=dict(
                                #color='MediumPurple',
                                width=0.5)
                            )))#,marker_color = "black"))
      fig.add_trace(go.Scatter(x=x_marker, y=y_marker
            ,showlegend=True
            ,name=name
            ,mode="markers"#"markers"
            ,marker_symbol = markers[data_idx]# + "-open-dot"
                        #,marker_color = "black"
                        #,marker_size=9
                        ,marker=dict(
                            color=colors[data_idx]#"black",#"red",#'rgba(135, 206, 250, 0.5)',
                            ,size=9
                            ,line=dict( 
                                #color='MediumPurple',
                                width=0.5)
                            )))#,marker_color = "black"))
   fig.update_layout(
                #title_text=title
                #,title_x=0.5
                #,title_y=1
                #xaxis_title="Turn"
                #,yaxis_title="Nodes"
                autosize=False
                ,width=700
                ,height=350
                ,plot_bgcolor='rgba(0,0,0,0)'
                ,legend=dict(
                    #title = "Formula",
                    #orientation="h",
                    #yanchor="top",
                    y=0,
                    xanchor="right",
                    x=1,  
                    font = dict(family = "Arial", size = 11, color = "black"),
                    #bordercolor="LightSteelBlue",
                    borderwidth=2,
                    itemsizing='trace',
                    #itemwidth = 30
                    )
                ,title={"text":plot_title}
                )
   fig["layout"]["yaxis"]["tickmode"] = "linear"
   fig["layout"]["yaxis"]["tick0"] = 4
   fig["layout"]["yaxis"]["dtick"] = 2
   fig["layout"]["yaxis"]["showgrid"] = True
   fig["layout"]["yaxis"]["gridcolor"] = "black"
   fig["layout"]["yaxis"]["gridwidth"] = 0.6
   fig["layout"]["xaxis"]["tickmode"] = "linear"
   fig["layout"]["xaxis"]["tick0"] = 0
   fig["layout"]["xaxis"]["dtick"] = 500
   fig["layout"]["xaxis"]["showgrid"] = True
   fig["layout"]["xaxis"]["gridcolor"] = "black"
   fig["layout"]["xaxis"]["gridwidth"] = 0.6
   fig['layout'].update(margin=dict(
                                       l=20,
                                       r=10,
                                       b=0,
                                       t=30))    
   fig.update_xaxes(range=[0,5000])
   fig.update_yaxes(range=[6,max_max_y])
   fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
   return fig

def get_subset(data, agent_name, f_index, c_param=None):
   temp_data = data.loc[data["f_index"]==f_index]
   if c_param is not None:
      temp_data = temp_data.loc[temp_data["c_param"]==c_param]
   #print(temp_data["player"].unique())
   temp_data = temp_data.loc[temp_data["player"] == agent_name]
   return temp_data

dummy_state = FunctionOptimisationState(players=[None], function=0, ranges=[[0,1]], splits=2)
functions = dummy_state.function_list
join = "/"
logs_path = "logs/Old/FO"
output_name = "collective_tree_logs.csv"
data = pd.read_csv(logs_path + join + "collective_tree_logs.csv")
n_buckets = 400
f_max_locations = [0.5,0.867,None,0.1,0.1]

for f_index in [0]:#,1,2,3,4]:
   agents_names = data["player"].unique()
   #print(agents_names)
   c_params = data["c_param"].unique()
   c_params.sort()
   generic_name = "MCTS_c"
   agents_names = [x for x in agents_names]
   names = []
   data_list = []

   #order names
   for c in c_params:
      string_c = str(c)
      if string_c.split(".")[-1] == "0":
         string_c = string_c[:-2]
      names.append(generic_name + string_c)
   for name in agents_names:
      if "SE_MCTS" in name:
         names.append(name)
   for name in names:
      temp_data = get_subset(data, name, f_index)
      data_list.append(temp_data)
   
   treated_names = []
   temp_names = [x for x in names]
   for it,st in enumerate(temp_names):
      new_string = st.replace("_c"," C = ")
      new_string = new_string.replace("1.414","sqrt(2)fred")
      new_string = new_string.split("fred")[0]
      if "SE_MCTS" in new_string:
         if "2600" in new_string:
            new_string = "SE_MCTS partial simulations"
            #continue
         else:
            new_string = "SIEA_MCTS complete simulations"
      treated_names.append(new_string)

   for i,df in enumerate(data_list):
      df['player'] = df['player'].replace([names[i]],treated_names[i])

   for i,name in enumerate(treated_names):
      if name == "SE_MCTS partial simulations":
         del data_list[i]
   
   #plot = exps.show_search(data_list, functions[f_index], "", 3, n_buckets = n_buckets, type="divisions")#, subplot_titles = treated_names+["Function "+str(f_index+1)], max_x_location=f_max_locations[f_index], y_ref_value=None)
   plot = show_search_depth(data_list, "Average expansion depth by iteration. Function " + str(f_index+1), 30, 10)
   #depth_plot.write_image(os.path.join(logs_path, "depth" + str(f_index) + '.png'), width=600, height=350)
   #plot = show_search2(data_list, functions[f_index], "", 3, n_buckets = 200, type="divisions", subplot_titles = agents_names+["Function "+str(f_index)])
   plot.show()
   #depth_plot.write_image(os.path.join(logs_path, "depth_f" + str(f_index) + '.png'), width=800, height=400)
   #plot.write_image(os.path.join(logs_path, "depth_histo_max_2k_f" + str(f_index) + '.png'), width=800, height=800)
   #plot.write_image(os.path.join(logs_path, "Average_depth_by_iteration_f" + str(f_index+1) + '.png'), width=800, height=400)
   